In [1]:
import torch
from train import train_model
from dataset.utils import *
from utils import *
import numpy as np

In [2]:
batch_size = 32
def redefine_train_parameters():
    global device, epochs, learning_rate, loss_lr, num_classes, embedding_size
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    epochs = 2
    learning_rate = 0.001
    loss_lr = 0.01
    num_classes = get_num_classes(train_dataset)
    embedding_size = 512

In [3]:
def train(train_dataset, test_dataset):
    train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
    test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
    model = train_model(model_name='resnet50',
                    loss_func_name='proxy_anchor',
                    num_classes=num_classes,
                    dataloader=train_loader,
                    epochs=epochs,
                    learning_rate=learning_rate,
                    verbose=0)
    return model, train_loader, test_loader

In [4]:
def cross_validate(dataset, k=10):
    global r1,r2,r4,r8,r16,r32
    k=k
    split_len = len(dataset)//k
    max_index = split_len*k
    r1,r2,r4,r8,r16,r32= [],[],[],[],[],[]
    for fold in range(k):
        print(f'''----------
fold {fold+1}:''')
        start = split_len * fold
        train_dataset = dataset[:start] + dataset[start + split_len:max_index]
        test_dataset = dataset[start:start+split_len]
        train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
        test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
        model = train_model(model_name='resnet50',
                        loss_func_name='proxy_anchor',
                        num_classes=num_classes,
                        dataloader=train_loader,
                        epochs=epochs,
                        learning_rate=learning_rate,
                        verbose=0)
        recall = evaluate_cos(model, test_loader)
        r1.append(recall[0])
        r2.append(recall[1])
        r4.append(recall[2])
        r8.append(recall[3])
        r16.append(recall[4])
        r32.append(recall[5])

In [5]:
def avg_recall(name):
    print(f'''Average Recall {name}:
R@1 : {np.mean(r1)*100:.4f}
R@2 : {np.mean(r2)*100:.4f}
R@4 : {np.mean(r4)*100:.4f}
R@8 : {np.mean(r8)*100:.4f}
R@16 : {np.mean(r16)*100:.4f}
R@32 : {np.mean(r32)*100:.4f}''')

In [81]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100)
redefine_train_parameters()

In [82]:
# PA
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 330.9441 Best loss: 8.7281


In [83]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [02:42<00:00,  1.09it/s]


R@1 : 7.177
R@2 : 10.355
R@4 : 14.908
R@8 : 19.497
R@16 : 24.656
R@32 : 30.834


100%|██████████| 262/262 [04:02<00:00,  1.08it/s]


R@1 : 1.646
R@2 : 2.374
R@4 : 3.400
R@8 : 4.509
R@16 : 5.857
R@32 : 7.575


[0.01646188715257068,
 0.02373851843015627,
 0.03399737564117858,
 0.04509125611356316,
 0.05857091733269713,
 0.07574853870929261]

In [84]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, flip=0.2)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100)
redefine_train_parameters()

In [85]:
# PA+Flip(.2)
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 331.5996 Best loss: 8.7856


In [86]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [02:42<00:00,  1.08it/s]


R@1 : 6.624
R@2 : 9.909
R@4 : 13.605
R@8 : 18.336
R@16 : 24.389
R@32 : 31.280


100%|██████████| 262/262 [04:02<00:00,  1.08it/s]


R@1 : 1.193
R@2 : 1.909
R@4 : 2.851
R@8 : 3.889
R@16 : 5.583
R@32 : 7.408


[0.011928903733746868,
 0.01908624597399499,
 0.028510079923655018,
 0.03888822617201479,
 0.05582726947393535,
 0.07407849218656805]

In [87]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, hist='v')
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, hist='v')
redefine_train_parameters()

In [88]:
# PA+AdaptiveV
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 418.6383 Best loss: 8.7858


In [89]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [03:25<00:00,  1.17s/it]


R@1 : 5.196
R@2 : 8.320
R@4 : 12.230
R@8 : 17.354
R@16 : 23.264
R@32 : 30.387


100%|██████████| 262/262 [05:06<00:00,  1.17s/it]


R@1 : 0.895
R@2 : 1.598
R@4 : 2.219
R@8 : 3.531
R@16 : 4.962
R@32 : 6.656


[0.008946677800310152,
 0.015984731003220804,
 0.022187760944769177,
 0.035309555051890734,
 0.049624239532386975,
 0.06656328283430753]

In [90]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median=True)
redefine_train_parameters()

In [91]:
# PA+Median
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 435.6178 Best loss: 8.7316


In [92]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [03:33<00:00,  1.21s/it]


R@1 : 6.106
R@2 : 9.052
R@4 : 12.569
R@8 : 18.015
R@16 : 23.585
R@32 : 30.334


100%|██████████| 262/262 [05:19<00:00,  1.22s/it]


R@1 : 0.811
R@2 : 1.264
R@4 : 2.195
R@8 : 3.483
R@16 : 4.831
R@32 : 6.776


[0.00811165453894787,
 0.012644637957771682,
 0.021949182870094237,
 0.034832398902540855,
 0.04831206012167482,
 0.06775617320768221]

In [93]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, hist='v', median_before=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, hist='v', median_before=True)
redefine_train_parameters()

In [94]:
# PA+Median+AdaptiveV
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 521.8434 Best loss: 8.7016


In [95]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [04:16<00:00,  1.46s/it]


R@1 : 6.356
R@2 : 9.766
R@4 : 14.640
R@8 : 19.336
R@16 : 24.585
R@32 : 31.405


100%|██████████| 262/262 [06:24<00:00,  1.47s/it]


R@1 : 1.264
R@2 : 1.944
R@4 : 3.006
R@8 : 4.056
R@16 : 5.452
R@32 : 7.110


[0.012644637957771682,
 0.019444113086007395,
 0.03006083740904211,
 0.040558272694739356,
 0.05451509006322319,
 0.07109626625313134]

In [96]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, unsharp=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, unsharp=True)
redefine_train_parameters()

In [97]:
# PA+Unsharp
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 380.3820 Best loss: 8.6672


In [98]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [03:06<00:00,  1.06s/it]


R@1 : 8.659
R@2 : 12.891
R@4 : 17.711
R@8 : 22.675
R@16 : 28.709
R@32 : 35.351


100%|██████████| 262/262 [04:38<00:00,  1.06s/it]


R@1 : 1.264
R@2 : 2.052
R@4 : 3.066
R@8 : 4.700
R@16 : 6.418
R@32 : 8.768


[0.012644637957771682,
 0.020517714422044614,
 0.030657282595729453,
 0.046999880710962665,
 0.06417750208755815,
 0.08767744244303949]

In [99]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, gaussian_blur=.5)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, gaussian_blur=.5)
redefine_train_parameters()

In [100]:
# PA+Gaussian(sigma=.5)
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 335.4895 Best loss: 8.7598


In [101]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [02:43<00:00,  1.07it/s]


R@1 : 4.535
R@2 : 7.338
R@4 : 11.123
R@8 : 16.783
R@16 : 23.210
R@32 : 30.370


100%|██████████| 262/262 [04:05<00:00,  1.07it/s]


R@1 : 0.692
R@2 : 1.193
R@4 : 1.813
R@8 : 2.911
R@16 : 4.438
R@32 : 6.644


[0.006918764165573184,
 0.011928903733746868,
 0.01813193367529524,
 0.029106525110342358,
 0.044375521889538354,
 0.06644399379697005]

In [102]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, unsharp=True, gaussian_blur=1.5)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, unsharp=True, gaussian_blur=1.5)
redefine_train_parameters()

In [103]:
# PA+Unsharp+Gaussian(sigma=1.5)
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 384.2512 Best loss: 8.8093


In [104]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [03:08<00:00,  1.07s/it]


R@1 : 5.374
R@2 : 8.498
R@4 : 12.194
R@8 : 16.729
R@16 : 21.835
R@32 : 28.406


100%|██████████| 262/262 [04:41<00:00,  1.08s/it]


R@1 : 1.634
R@2 : 2.445
R@4 : 3.376
R@8 : 4.760
R@16 : 6.215
R@32 : 8.004


[0.01634259811523321,
 0.02445425265418108,
 0.033758797566503636,
 0.047596325897650005,
 0.06214958845282119,
 0.08004294405344149]

In [105]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, unsharp=True, unsharp_radius=5, unsharp_amount=2, gaussian_blur=.5)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, unsharp=True, unsharp_radius=5, unsharp_amount=2, gaussian_blur=.5)
redefine_train_parameters()

In [106]:
# PA+Unsharp(radius=5,amount=2)+Gaussian(sigma=.5)
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 348.0265 Best loss: 8.7263


In [107]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [02:50<00:00,  1.03it/s]


R@1 : 6.070
R@2 : 9.177
R@4 : 13.676
R@8 : 18.532
R@16 : 24.978
R@32 : 32.262


100%|██████████| 262/262 [04:14<00:00,  1.03it/s]


R@1 : 1.014
R@2 : 1.646
R@4 : 2.601
R@8 : 3.567
R@16 : 5.141
R@32 : 7.324


[0.010139568173684839,
 0.01646188715257068,
 0.026005010139568175,
 0.03566742216390314,
 0.051413575092449,
 0.07324346892520578]

In [108]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median_unsharp=True, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median=True)
redefine_train_parameters()

In [109]:
# PA+MedianUnsharp+Median
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 496.4664 Best loss: 8.7183


In [110]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [04:03<00:00,  1.39s/it]


R@1 : 6.035
R@2 : 9.998
R@4 : 15.158
R@8 : 21.139
R@16 : 27.959
R@32 : 35.976


100%|██████████| 262/262 [06:04<00:00,  1.39s/it]


R@1 : 2.183
R@2 : 3.507
R@4 : 5.356
R@8 : 8.088
R@16 : 11.082
R@32 : 14.661


[0.02182989383275677,
 0.03507097697721579,
 0.05356077776452344,
 0.08087796731480378,
 0.11081951568650841,
 0.14660622688774902]

In [6]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median_unsharp=True, median_unsharp_amount=2, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median_unsharp_amount=2, median=True)
redefine_train_parameters()

In [7]:
# PA+MedianUnsharp(amount=2)+Median
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 502.4112 Best loss: 8.7193


In [8]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [04:06<00:00,  1.40s/it]


R@1 : 4.071
R@2 : 6.588
R@4 : 10.141
R@8 : 14.515
R@16 : 20.104
R@32 : 26.834


100%|██████████| 262/262 [06:08<00:00,  1.41s/it]


R@1 : 0.406
R@2 : 0.787
R@4 : 1.479
R@8 : 2.410
R@16 : 3.722
R@32 : 5.404


[0.004055827269473935,
 0.007873076464272934,
 0.014791840629846117,
 0.024096385542168676,
 0.03721817964929023,
 0.054037933913873314]

In [111]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median_unsharp=True, median_unsharp_amount=5, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median_unsharp_amount=5, median=True)
redefine_train_parameters()

In [112]:
# PA+MedianUnsharp(amount=5)+Median
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 502.3842 Best loss: 8.7486


In [113]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [04:06<00:00,  1.40s/it]


R@1 : 6.320
R@2 : 9.302
R@4 : 13.587
R@8 : 18.800
R@16 : 25.174
R@32 : 32.244


100%|██████████| 262/262 [06:09<00:00,  1.41s/it]


R@1 : 0.811
R@2 : 1.300
R@4 : 1.956
R@8 : 3.066
R@16 : 4.807
R@32 : 6.955


[0.00811165453894787,
 0.013002505069784088,
 0.019563402123344866,
 0.030657282595729453,
 0.048073482046999884,
 0.06954550876774425]

In [114]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, flip=.2, median_unsharp=True, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median=True)
redefine_train_parameters()

In [115]:
# PA+Flip(.2)+MedianUnsharp+Median
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 495.9452 Best loss: 8.7392


In [116]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [04:03<00:00,  1.39s/it]


R@1 : 5.071
R@2 : 7.445
R@4 : 10.962
R@8 : 15.479
R@16 : 21.514
R@32 : 28.156


100%|██████████| 262/262 [06:04<00:00,  1.39s/it]


R@1 : 0.823
R@2 : 1.408
R@4 : 2.147
R@8 : 3.316
R@16 : 4.962
R@32 : 6.728


[0.00823094357628534,
 0.014076106405821305,
 0.021472026720744365,
 0.033162352379816296,
 0.049624239532386975,
 0.06727901705833234]

In [117]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median_unsharp=True, median_unsharp_amount=5, median=True, flip=.2)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median_unsharp_amount=5, median=True)
redefine_train_parameters()

In [118]:
# PA+Flip(.2)+MedianUnsharp(amount=5)+Median
model, train_loader, test_loader = train(train_dataset,test_dataset)

Finished Training, Time: 502.3705 Best loss: 8.6946


In [119]:
evaluate_cos(model,train_loader)
evaluate_cos(model,test_loader)

100%|██████████| 176/176 [04:10<00:00,  1.42s/it]


R@1 : 5.071
R@2 : 8.106
R@4 : 12.284
R@8 : 17.533
R@16 : 23.960
R@32 : 31.369


100%|██████████| 262/262 [06:12<00:00,  1.42s/it]


R@1 : 0.930
R@2 : 1.598
R@4 : 2.553
R@8 : 3.662
R@16 : 5.213
R@32 : 7.575


[0.009304544912322557,
 0.015984731003220804,
 0.0255278539902183,
 0.03662173446260289,
 0.05212930931647382,
 0.07574853870929261]